In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Vytvorenie klienta

V tomto príklade použijeme [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) na prístup k LLM.

`model` je definovaný ako `gpt-4o-mini`. Skúste zmeniť model na iný dostupný na trhu GitHub Models, aby ste videli rôzne výsledky.

Ako rýchly test jednoducho spustíme jednoduchý príkaz - `Aké je hlavné mesto Francúzska`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definovanie funkcií

V tomto príklade poskytneme agentovi prístup k nástroju, ktorý je funkciou s zoznamom dostupných dovolenkových destinácií a ich dostupnosťou.

Môžete si predstaviť, že ide o scenár, kde by cestovný agent mohol mať prístup k cestovnej databáze, napríklad.

Keď budete prechádzať týmto príkladom, neváhajte skúsiť definovať nové funkcie a nástroje, ktoré môže agent použiť.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definovanie nástroja funkcie
Aby agent mohol používať `vacation_destinations` ako `FunctionTool`, musíme ho definovať ako taký.

Poskytneme tiež popis nástroja, ktorý pomôže agentovi identifikovať, na čo sa tento nástroj používa v súvislosti s úlohou, ktorú používateľ požaduje.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definovanie agenta

Teraz môžeme vytvoriť agenta v nasledujúcom kóde. Definujeme `system_message`, aby sme agentovi poskytli pokyny, ako pomôcť používateľom nájsť dovolenkové destinácie.

Taktiež nastavíme parameter `reflect_on_tool_use` na hodnotu true. To umožňuje LLM vziať odpoveď z volania nástroja a poslať odpoveď pomocou prirodzeného jazyka.

Môžete nastaviť parameter na hodnotu false, aby ste videli rozdiel.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Spustenie agenta

Teraz môžeme spustiť agenta s počiatočnou správou od používateľa, ktorý sa pýta na výlet do Tokia.

Môžete zmeniť túto cieľovú destináciu, aby ste videli, ako agent reaguje na dostupnosť mesta.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
